In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import numpy as np

In [5]:
# all data
traindf=pd.read_csv("./data/train.csv")
validationdf = pd.read_csv("./data/validation.csv")

C:\Users\krish\AppData\Local\Temp\ipykernel_15552\3123182197.py:2: DtypeWarning: Columns (4,13,14,16,47) have mixed types. Specify dtype option on import or set low_memory=False.
  traindf=pd.read_csv("./data/train.csv")
C:\Users\krish\AppData\Local\Temp\ipykernel_15552\3123182197.py:3: DtypeWarning: Columns (13,16,36) have mixed types. Specify dtype option on import or set low_memory=False.
  validationdf = pd.read_csv("./data/validation.csv")


In [6]:
traindf.replace('\\N', None, inplace=True)

# dropping values that are not neeeded
df=traindf.drop(['time_x', 'timetaken_in_millisec', 'fastestLap', 'rank', 'fastestLapTime', 'max_speed', 'time_y', 'url_x', 'fp1_date', 'fp1_time', 'fp2_date'
            , 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'driver_num', 'driver_code', 'url_y'
             ,"number", "position_x", 'nationality_y', 'url', 'status', 'positionText_y', 'constructorRef', 'forename', 'surname', 'driverRef', 'positionText_x', 'nationality'], axis=1)


#converting the dob and date column into years to determine the age of the driver
df['dob'] = pd.to_datetime(df['dob'])
df['date'] = pd.to_datetime(df['date'])
df['time_difference'] = df['date'] - df['dob']
df['years'] = (df['time_difference'].dt.days / 365.25).astype(int)

df = df.drop(['dob', 'date', 'time_difference'], axis=1)

# using frequency encoding for grand_prix and company
frequency_grand_prix = df['grand_prix'].value_counts(normalize=True)
df['grand_prix_encoded'] = df['grand_prix'].map(frequency_grand_prix)

frequency_company = df['company'].value_counts(normalize=True)
df['company_encoded'] = df['company'].map(frequency_company)

df = df.drop(['grand_prix', 'company'], axis=1)

In [7]:
validationdf.replace('\\N', None, inplace=True)

# dropping values that are not neeeded
df2=validationdf.drop(['time_x', 'timetaken_in_millisec', 'fastestLap', 'rank', 'fastestLapTime', 'max_speed', 'time_y', 'url_x', 'fp1_date', 'fp1_time', 'fp2_date'
            , 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'driver_num', 'driver_code', 'url_y'
             ,"number", "position_x", 'nationality_y', 'url', 'status', 'positionText_y', 'constructorRef', 'forename', 'surname', 'driverRef', 'positionText_x', 'nationality'], axis=1)


#converting the dob and date column into years to determine the age of the driver
df2['dob'] = pd.to_datetime(df2['dob'])
df2['date'] = pd.to_datetime(df2['date'])
df2['time_difference'] = df2['date'] - df2['dob']
df2['years'] = (df2['time_difference'].dt.days / 365.25).astype(int)

df2 = df2.drop(['dob', 'date', 'time_difference'], axis=1)

# using frequency encoding for grand_prix and company
frequency_grand_prix = df2['grand_prix'].value_counts(normalize=True)
df2['grand_prix_encoded'] = df2['grand_prix'].map(frequency_grand_prix)

frequency_company = df2['company'].value_counts(normalize=True)
df2['company_encoded'] = df2['company'].map(frequency_company)

df2 = df2.drop(['grand_prix', 'company'], axis=1)

In [8]:
df = df.drop(['grand_prix_encoded'], axis=1)
df2 = df2.drop(['grand_prix_encoded'], axis=1)

In [22]:
X_train = df.drop(columns=['position', 'result_driver_standing'])
y_train = df['position']
train_id = df['result_driver_standing']

X_test = df2.drop(columns=['position', 'result_driver_standing'])
y_test = df2['position']
test_id = df2['result_driver_standing']


In [32]:
X_train.apply(pd.to_numeric)
X_test.apply(pd.to_numeric)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [97]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(18, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = x.view(-1, 18)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x        

In [120]:
model = SimpleNN().to(device)

train_data = torch.tensor(X_train.values, dtype=torch.float32).to(device)
test_data = torch.tensor(X_test.values, dtype=torch.float32).to(device)
train_targets = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1).to(device)
test_targets = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1).to(device)

In [99]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()

    def forward(self, y_pred, y_true):
        mse = nn.MSELoss()
        return torch.sqrt(mse(y_pred, y_true))

In [121]:

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [78]:
num_epochs = 10  # Example: Train for a maximum of 100 epochs
best_loss = float('inf')
best_model_state = None
patience = 5  # Stop if validation loss does not improve for 5 consecutive epochs
counter = 0  # Counter to track epochs without improvement

In [122]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()                         # Set the model to training mode
    optimizer.zero_grad()
    outputs = model.forward(train_data)
    
    loss = criterion(outputs, train_targets)  # Compute RMSE loss for all output nodes
    
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 99021.6406
Epoch [2/100], Loss: 20487.7051
Epoch [3/100], Loss: 21677.5723
Epoch [4/100], Loss: 8172.7744
Epoch [5/100], Loss: 2578.7578
Epoch [6/100], Loss: 240.9350
Epoch [7/100], Loss: 191.2259
Epoch [8/100], Loss: 168.6819
Epoch [9/100], Loss: 151.0499
Epoch [10/100], Loss: 139.1486
Epoch [11/100], Loss: 136.4941
Epoch [12/100], Loss: 143.9600
Epoch [13/100], Loss: 152.9803
Epoch [14/100], Loss: 158.5436
Epoch [15/100], Loss: 159.1911
Epoch [16/100], Loss: 189.8098
Epoch [17/100], Loss: 157.9162
Epoch [18/100], Loss: 154.9705
Epoch [19/100], Loss: 149.6794
Epoch [20/100], Loss: 143.6344
Epoch [21/100], Loss: 138.3233
Epoch [22/100], Loss: 134.8961
Epoch [23/100], Loss: 134.0976
Epoch [24/100], Loss: 135.6194
Epoch [25/100], Loss: 138.3990
Epoch [26/100], Loss: 140.6029
Epoch [27/100], Loss: 140.6772
Epoch [28/100], Loss: 137.7447
Epoch [29/100], Loss: 132.2056
Epoch [30/100], Loss: 126.6808
Epoch [31/100], Loss: 124.7991
Epoch [32/100], Loss: 126.9750
Epoch [33

In [38]:
import math
from sklearn.metrics import mean_squared_error


In [124]:
model.eval()                             # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(test_data)
    rmse = math.sqrt(mean_squared_error(test_targets.cpu(), outputs.cpu()))
    print(f'RMSE on Validation Set: {rmse:.4f}')

RMSE on Validation Set: 7.8239
